情緒辭典參考 http://nlp.innobic.yzu.edu.tw/resources/cvaw.html

In [1]:
import pandas as pd
import numpy as np 
import re

In [9]:
pd.set_option('display.max_rows',200)
pd.set_option('display.max_columns',200)
pd.set_option('display.max_colwidth',200)

In [10]:
df = pd.read_csv('hahow_feedback.csv', delimiter=',',encoding='ANSI')

In [11]:
df.head(3)

,文本,情緒標記
0,上完課後，真的很慶幸有買到課程並真心覺得物超所值，對於新手的我，淺顯易懂很容易就了解製作podcast的技巧，一直都很喜歡聽podcast，但沒想到可以自己創作出自己的節目來，能接觸到這個課程真的很開心，課程適合新手入門，講了很多製作podcast的基本原則，能夠打好基礎之後再自己慢慢精進能力，對於自己經營製作節目，或是網路行銷的概念都很實用，上課不會無聊想睡覺，每一節課長度剛剛好節奏明快...,1.0
1,"是很值得的課, 對入門很有用",1.0
2,很棒的課，獲益良多！謝謝老師。,1.0


In [44]:
df[df.index==8]['文本'].values[0]

'完整的內容與清晰的課程，讓人明確知道如何實踐。'

In [59]:
def get_raw_txtLen(txt):
    return len(txt)

## 選取字數超過40字的文本

In [77]:
df_selected = df[df['文本'].apply(get_raw_txtLen)>40]

In [78]:
df_selected.tail(3)

,文本,情緒標記
79,原本對這堂課很期待..結果老師感覺根本在照本宣科..內容也很貧脊..感覺去找本給初學者的暢銷...,0.0
80,原本以為老師會分享他獨有的進出場方法，但看完根本只是上網找些資料，逐字稿的照唸，而且每集的錄...,0.0
85,"畫面與聲音""貌合神離""，像在聽廣播，沒有發揮影音教學的特質，到最後解鎖課程時，才說明課程外包...",0.0


---

# 斷句

In [70]:
def cut2sentence(txt):
    txt = txt.split('，')
    return txt

### 針對第一個文本斷句結果

In [80]:
df_selected['文本'].apply(cut2sentence)[0]

['上完課後',
 '真的很慶幸有買到課程並真心覺得物超所值',
 '對於新手的我',
 '淺顯易懂很容易就了解製作podcast的技巧',
 '一直都很喜歡聽podcast',
 '但沒想到可以自己創作出自己的節目來',
 '能接觸到這個課程真的很開心',
 '課程適合新手入門',
 '講了很多製作podcast的基本原則',
 '能夠打好基礎之後再自己慢慢精進能力',
 '對於自己經營製作節目',
 '或是網路行銷的概念都很實用',
 '上課不會無聊想睡覺',
 '每一節課長度剛剛好節奏明快',
 '此外',
 '老師也很熱心給予建議',
 '讓學生能得到實質上的進步',
 '謝謝老師的用心！']

### 針對特殊情況

In [82]:
df_selected['文本'].apply(cut2sentence)[79]

['原本對這堂課很期待..結果老師感覺根本在照本宣科..內容也很貧脊..感覺去找本給初學者的暢銷書都可以講得比這有趣...非常不推薦購買...(有種浪費錢的感覺)']

In [91]:
special_txt=df_selected['文本'].apply(cut2sentence)[79]

pattern = re.sub('(\…{2})([^”’])', r"\1\n\2", special_txt[0])

In [109]:
[stc for stc in special_txt[0].replace('.',' ').split(' ') if stc != '']

['原本對這堂課很期待',
 '結果老師感覺根本在照本宣科',
 '內容也很貧脊',
 '感覺去找本給初學者的暢銷書都可以講得比這有趣',
 '非常不推薦購買',
 '(有種浪費錢的感覺)']

---

# 文本相似度

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import jieba

In [99]:
corpus= [
    '這堂課很糟糕',
    '我覺得這堂課很糟糕',
    '老師講話很慢好想睡覺',
    '有點想睡覺',
    '有點想睡覺',
    '感覺去找本給初學者的暢銷書都可以講得比這有趣',
    '結果老師感覺根本在照本宣科'
]

In [100]:
corpus_cleaned= []
for txt in corpus:
    words_cut = jieba.cut(txt)
    corpus_cleaned.append(' '.join(words_cut))

In [101]:
corpus_cleaned

['這堂 課 很 糟糕',
 '我覺 得 這堂 課 很 糟糕',
 '老師 講話 很慢 好 想 睡覺',
 '有點 想 睡覺',
 '有點 想 睡覺',
 '感覺 去 找 本給 初學者 的 暢銷 書 都 可以 講得 比 這 有趣',
 '結果 老師 感覺 根本 在 照本宣科']

In [102]:
vectorizer = TfidfVectorizer(min_df=0, token_pattern='\\b\\w+\\b')
tfidf_matrix = vectorizer.fit_transform(corpus_cleaned)

# BoW

In [103]:
bag_of_words = vectorizer.get_feature_names()
print(bag_of_words)

['初學者', '去', '可以', '在', '好', '很', '很慢', '得', '想', '感覺', '我覺', '找', '暢銷', '書', '有趣', '有點', '本給', '根本', '比', '照本宣科', '的', '睡覺', '糟糕', '結果', '老師', '課', '講得', '講話', '這', '這堂', '都']


In [104]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [105]:
best_idx = np.argmax(cosine_similarities[2])

# 文本相似度

In [109]:
for idx ,txt in enumerate(corpus_cleaned): 
    best_match = cosine_similarities[idx].argsort()[-2]
    
    print('文本 idx:{} 內容: {}'.format(idx, corpus[idx]))
    print('最相近的文本: {}\n'.format(corpus[best_match]))

文本 idx:0 內容: 這堂課很糟糕
最相近的文本: 我覺得這堂課很糟糕

文本 idx:1 內容: 我覺得這堂課很糟糕
最相近的文本: 這堂課很糟糕

文本 idx:2 內容: 老師講話很慢好想睡覺
最相近的文本: 有點想睡覺

文本 idx:3 內容: 有點想睡覺
最相近的文本: 有點想睡覺

文本 idx:4 內容: 有點想睡覺
最相近的文本: 有點想睡覺

文本 idx:5 內容: 感覺去找本給初學者的暢銷書都可以講得比這有趣
最相近的文本: 結果老師感覺根本在照本宣科

文本 idx:6 內容: 結果老師感覺根本在照本宣科
最相近的文本: 老師講話很慢好想睡覺

